# 3.3. Limpieza y preparación de datos.

In [ ]:
import pandas as pd
import numpy as np

## Tratamiento de datos en blanco (<i>missing values</i>)

- En la mayoría de los ficheros utilizados como fuente de datos, es muy común la existencia de valores nulos (en blanco, <i>missing</i>...). 
- Estos "huecos" en la información suelen ser muy problemáticos, ya que tiene un impacto importante a la hora de realizar cualquier tipo de cálculo numérico y son difícilmente interpretables.
- Uno de los objetivos de pandas es facilitar el tratamiento de este tipo de datos no existentes, ofreciendo múltiples funciones que permiten llevar a cabo tanto su detección, como su eliminación o imputación...

#### Detección de <i>missing values</i>

Pandas ofrece principalmente dos funciones para manejar la detección de valores nulos.<br/>
<ul>
<li><b>isnull:</b> Que devuelve una Serie o DataFrame booleano indicando qué elemetos son NaN o None.</li>
<li><b>notnull:</b> Que devuelve el inverso del anterior.</li>

In [ ]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

In [ ]:
string_data.isnull()

In [ ]:
string_data[0] = None
string_data

In [ ]:
string_data.isnull()

#### Eliminación de registros con <i>missing values</i>

- Conviene hacer un estudio cuidadoso del por qué y la casuística de los valores nulos.
- Uno de los posibles tratamientos a aplicar es su eliminación directa del set de datos.
- Pandas, nos ofrece el método <b>dropna</b> para llevar a cabo esta tarea.
- Eliminando toda la fila o columna que tenga un missing value
- Los parámetros de este método son:<br/>
<ul>
<li><b>axis:</b> Selección de eje sobre el que realizar la eliminación.</li>
<li><b>how:</b> Tomará posibles valores 'any' y 'all' e indica si se debe eliminar la fila o columna cuando haya uno o más valores NaN o cuando todos los valores sean NaN.</li>
<li><b>thresh:</b> Permite indicar, el número de observaciones no nulas que se deben tener para no realizar el borrado.</li>
</ul>

In [ ]:
from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])
data

In [ ]:
data.dropna()

In [ ]:
data[data.notnull()]

In [ ]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])
data

In [ ]:
cleaned = data.dropna()
cleaned

In [ ]:
data.dropna(how='all')

In [ ]:
data[4] = NA
data

In [ ]:
data.dropna(axis=1, how='all')

In [ ]:
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

In [ ]:
df.dropna()

In [ ]:
df.dropna(thresh=2)

#### Relleno de registros con <i>missing values</i>

- Existirán casos en los que no se desee (o no se pueda) eliminar los registros con valores nulos (p.e. podrían suponer un porcentaje demasiado elevado de nuestro set de datos). 
- En estos casos, habrá que realizar una imputación de los mismos a un valor preestablecidor.
- Para esta tarea Pandas incorpora el método <b>fillna</b>, que cuenta con los siguientes parámetros:<br/>
<ul>
<li><b>axis:</b> Que decide si aplicará el criterio de relleno por filas o columnas.</li>
<li><b>value:</b> Que rellena los valores nulos a un valor fijo.</li>
<li><b>method:</b> Que permitirá establecer un criterio de relleno de entre los siguientes:
<ul>
<li>ffill: Relleno en base a la observación de los últimos elementos no nulos.</li>
<li>bfill: Relleno en base a la observación de los próximos elementos no nulos.</li>
</ul>
<li><b>limit:</b> Contador máximo de elmentos imputados.</li>
</ul>

In [ ]:
df

In [ ]:
df.fillna(0)

In [ ]:
# Dictionario con el valo para cada columna
df.fillna({1: 0.5, 2: 0})

In [ ]:
# Puede ser inplace
_ = df.fillna(0, inplace=True)
df

In [ ]:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

In [ ]:
df.fillna(method='ffill')

In [ ]:
df.fillna(method='ffill', limit=2)

In [ ]:
data = pd.Series([1., NA, 3.5, NA, 7])
data

In [ ]:
data.fillna(data.mean())

### Eliminación de duplicados

In [ ]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

In [ ]:
data.duplicated()

In [ ]:
data.drop_duplicates()

In [ ]:
data['v1'] = range(7)
data

In [ ]:
data.drop_duplicates(['k1'])

In [ ]:
data.drop_duplicates(['k1', 'k2'], keep='last')

### Remplazamiento de valores

In [ ]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

In [ ]:
data.replace(-999, np.nan)

In [ ]:
data.replace([-999, -1000], np.nan)

In [ ]:
data.replace([-999, -1000], [np.nan, 0])

In [ ]:
# podemos pasarle un diccionario
data.replace({-999: np.nan, -1000: 0})

### Renombrado de índices y columnas

In [ ]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

In [ ]:
transform = lambda x: x[:4].upper()
data.index.map(transform)

In [ ]:
data.index = data.index.map(transform)
data

In [ ]:
data.rename(index=str.title, columns=str.upper)

In [ ]:
data

In [ ]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

In [ ]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

## Transformación de los datos aplicación de funciones sobre estructuras

- Pandas tiene un conjunto de funciones que  permiten aplicar operaciones elemento a elemento (o fila a fila, o columna a columna) en sus estructuras de datos. 

#### Aplicación de funciones elemento a elemento sobre Series - Función map

In [ ]:
serie = pd.Series([1, 2, 3, 4, 5, 6])
serie

In [ ]:
def es_par(elemento):
    if elemento % 2 == 0:
        return 'Par: ' + str(elemento)
    else:
        return 'Impar: ' + str(elemento)

In [ ]:
serie.map(es_par)

#### Aplicación de funciones elemento a elemento sobre Dataframe - Función map

In [ ]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

In [ ]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

In [ ]:
lowercased = data['food'].str.lower()
lowercased

In [ ]:
data['animal'] = lowercased.map(meat_to_animal)
data

In [ ]:
# podemos usar una función lambda y indexar el dict
data['food'].map(lambda x: meat_to_animal[x.lower()])

#### Aplicación de funciones elemento a elemento sobre DataFrames - Función applymap

In [ ]:
dataframe = pd.DataFrame(np.arange(16).reshape(4, 4))
dataframe

In [ ]:
def es_par(elemento):
    if elemento % 2 == 0:
        return 'Par: ' + str(elemento)
    else:
        return 'Impar: ' + str(elemento)

In [ ]:
dataframe.applymap(es_par)  

#### Aplicación de funciones fila a fila o columna a columna sobre DataFrames - Función apply

In [ ]:
dataframe = pd.DataFrame(np.arange(20).reshape(4, 5))
dataframe

In [ ]:
def es_suma_par(elemento):
    if np.sum(elemento) % 2 == 0:
        return 'Suma par: ' + str(np.sum(elemento))
    else:
        return 'Suma impar: ' + str(np.sum(elemento))

In [ ]:
dataframe.apply(es_suma_par, axis=0)

In [ ]:
dataframe.apply(es_suma_par, axis=1)

### Detección y filtrado de outliers

In [ ]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

In [ ]:
col = data[2]
col[np.abs(col) > 3]

In [ ]:
cond = (np.abs(data) > 3)
cond

In [ ]:
cond.any(axis=1)

In [ ]:
data[(np.abs(data) > 3).any(axis=1)]

In [ ]:
# acotamos a 3 y -3
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()